In [3]:
import numpy as np

def _pairwise_distances(X):
    """Compute squared Euclidean distances for X (n_samples, n_features)"""
    sum_X = np.sum(np.square(X), axis=1)
    D = np.add.outer(sum_X, sum_X) - 2 * np.dot(X, X.T)
    return D

def _hbeta(D, beta=1.0):
    """Compute entropy and probability for a given precision (beta)."""
    P = np.exp(-D * beta)
    sumP = np.sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

def _binary_search_perplexity(Di, desired_perplexity, tol=1e-5, max_iter=50):
    """Find beta for each row so that perplexity matches the desired value."""
    beta_min, beta_max = -np.inf, np.inf
    beta = 1.0
    logU = np.log(desired_perplexity)
    for i in range(max_iter):
        H, thisP = _hbeta(Di, beta)
        Hdiff = H - logU
        if np.abs(Hdiff) < tol:
            break
        if Hdiff > 0:
            beta_min = beta
            if beta_max == np.inf or beta_max == -np.inf:
                beta *= 2
            else:
                beta = (beta + beta_max) / 2
        else:
            beta_max = beta
            if beta_min == np.inf or beta_min == -np.inf:
                beta /= 2
            else:
                beta = (beta + beta_min) / 2
    return thisP

class TSNE:
    """
    t-SNE (t-distributed Stochastic Neighbor Embedding) for dimensionality reduction.
    """
    def __init__(self, n_components=2, perplexity=30.0, learning_rate=200.0, n_iter=1000):
        self.n_components = n_components
        self.perplexity = perplexity
        self.learning_rate = learning_rate
        self.n_iter = n_iter

    def _compute_joint_probabilities(self, X):
        N = X.shape[0]
        D = _pairwise_distances(X)
        P = np.zeros((N, N))
        for i in range(N):
            Di = np.delete(D[i], i)
            Pi = _binary_search_perplexity(Di, self.perplexity)
            P[i, np.arange(N) != i] = Pi
        # Symmetrize and normalize
        P = (P + P.T) / (2 * N)
        P = np.maximum(P, 1e-12)
        return P

    def fit_transform(self, X):
        np.random.seed(42)
        N = X.shape[0]
        Y = np.random.randn(N, self.n_components) * 1e-4

        P = self._compute_joint_probabilities(X)
        P *= 4.  # Early exaggeration
        momentum = 0.5
        min_gain = 0.01
        gains = np.ones_like(Y)
        Y_inc = np.zeros_like(Y)

        for it in range(self.n_iter):
            sum_Y = np.sum(np.square(Y), axis=1)
            num = 1 / (1 + np.add.outer(sum_Y, sum_Y) - 2 * np.dot(Y, Y.T))
            np.fill_diagonal(num, 0)
            Q = num / np.sum(num)
            Q = np.maximum(Q, 1e-12)

            PQ = P - Q
            for i in range(N):
                dY = 4 * np.sum((PQ[:, i][:, np.newaxis] * (Y[i] - Y)), axis=0)
                gains[i] = (gains[i] + 0.2) * ((np.sign(dY) != np.sign(Y_inc[i])) | (gains[i] < min_gain)) + \
                           (gains[i] * 0.8) * ((np.sign(dY) == np.sign(Y_inc[i])) & (gains[i] >= min_gain))
                gains[i] = np.maximum(gains[i], min_gain)
                Y_inc[i] = momentum * Y_inc[i] - self.learning_rate * gains[i] * dY
                Y[i] += Y_inc[i]

            if it == 250:
                P /= 4.
            if (it + 1) % 100 == 0 or it == 0:
                C = np.sum(P * np.log(P / Q))
                print(f"Iteration {it+1}: error = {C:.4f}")
            if it == 250:
                momentum = 0.8
        return Y

> ## Example usage:

In [4]:
# Create some sample data (two clusters in 3D)
np.random.seed(0)
X1 = np.random.randn(50, 3)
X2 = np.random.randn(50, 3) + 5
X = np.vstack([X1, X2])

tsne = TSNE(n_components=2, perplexity=10.0, learning_rate=100.0, n_iter=300)
X_embedded = tsne.fit_transform(X)
print("t-SNE embedding:\n", X_embedded)

Iteration 1: error = 14.2532


C:\Users\Adil\AppData\Local\Temp\ipykernel_8224\3194847720.py:78: RuntimeWarning: divide by zero encountered in divide
  num = 1 / (1 + np.add.outer(sum_Y, sum_Y) - 2 * np.dot(Y, Y.T))
C:\Users\Adil\AppData\Local\Temp\ipykernel_8224\3194847720.py:77: RuntimeWarning: overflow encountered in square
  sum_Y = np.sum(np.square(Y), axis=1)
C:\Users\Adil\AppData\Local\Temp\ipykernel_8224\3194847720.py:78: RuntimeWarning: overflow encountered in add
  num = 1 / (1 + np.add.outer(sum_Y, sum_Y) - 2 * np.dot(Y, Y.T))
C:\Users\Adil\AppData\Local\Temp\ipykernel_8224\3194847720.py:78: RuntimeWarning: overflow encountered in multiply
  num = 1 / (1 + np.add.outer(sum_Y, sum_Y) - 2 * np.dot(Y, Y.T))
C:\Users\Adil\AppData\Local\Temp\ipykernel_8224\3194847720.py:78: RuntimeWarning: invalid value encountered in subtract
  num = 1 / (1 + np.add.outer(sum_Y, sum_Y) - 2 * np.dot(Y, Y.T))


Iteration 100: error = nan
Iteration 200: error = nan
Iteration 300: error = nan
t-SNE embedding:
 [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]